In [1]:
import json

with open('block-groups-polygons-simple.geojson', 'r') as f1:
    js_blocks = json.load(f1)
        
with open('station.json', 'r') as f2:
    js_stations = json.load(f2)

In [2]:
def indexZones(shapeFilename):
    import rtree
    import fiona.crs
    import geopandas as gpd
    index = rtree.Rtree()
    zones = gpd.read_file(shapeFilename).to_crs(fiona.crs.from_epsg(2263))
    for idx,geometry in enumerate(zones.geometry):
        index.insert(idx, geometry.bounds)
    return (index, zones)

def findBlock(p, index, zones):
    match = index.intersection((p.x, p.y, p.x, p.y))
    for idx in match:
        if any(map(lambda x: x.contains(p), zones.geometry[idx])):
            return zones['properties'][idx]
    return -1

In [52]:
def mapToZone(stations):
    import pyproj
    import shapely.geometry as geom
    proj = pyproj.Proj(init="epsg:2263", preserve_units=True)    
    index, zones = indexZones('block-groups-polygons-simple.geojson')
    for station in stations:
        location = geom.Point(proj(station['latitude'], station['longitude']))
        zone = findBlock(location, index, zones)
        yield (station[stationName], zone)

#output = js_stations['stationBeanList'].map(mapToZone)

lines = sc.textFile('station.json')
data = lines.map(json.loads)
#d = sc.parallelize(js_stations['stationBeanList']).flatMap(lambda x: x.items())


# sc.parallelize(rdd.take(1))
# sc.parallelize(mapToZone(rdd.map(json.loads).collect()))
